In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    f1_score, accuracy_score, precision_score, recall_score, roc_auc_score,
    classification_report, make_scorer, confusion_matrix,
    roc_curve, precision_recall_curve, average_precision_score
)
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

# 📂 1. 데이터 불러오기
df = pd.read_csv('../data/WA_Fn-UseC_-HR-Employee-Attrition.csv')

# 🎯 2. 타겟 인코딩
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})

# 🔄 3. 범주형 → 원핫 인코딩
df = pd.get_dummies(df, drop_first=True)

# ✅ 4. 주요 13개 변수 선택
selected_columns = [
    'Age', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel',
    'JobSatisfaction', 'MonthlyIncome', 'OverTime_Yes', 'StockOptionLevel',
    'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole',
    'YearsWithCurrManager', 'DistanceFromHome'  # 추가 변수 1개 (총 13개)
]

X = df[selected_columns]
y = df['Attrition']

# ⚖️ 5. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# KNN 알고리즘은 거리기반
# Income이 숫자가 커서 Age 보다 영향력이 훨씬 커짐
# KNN은 거리에 민감하므로, 스케일을 맞춰줘야 공정하게 반영
# LogisticRegression, SVM, KMeans, PCA 등도 거리기반 또는 경사하강법 기반이므로 스케일링 중요
# 단, Tree 계열(RandomForest, XGBoost)은 스케일에 영향없음


# ⚖️ 6. SMOTE 적용
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)
# 불균형 클래스 문제를 해결하기 위한 오버샘플링 기법
# 소수 클래스의 데이터를 인위적으로 생성하여 데이터 셋의 불균형문제를 해결



# 📊 7. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# 🤖 8. 모델 학습
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# max_iter = 1000
# 경사하강 법으로 최적해를 찾을 때의 최대 반복수 (default = 100)
# 데이터가 크거나 수렴이 느리면 값을 키워야 함

# # 🔍 9. 예측
y_pred = model.predict(X_test)
# 테스트 데이터(X_test)에 대해 모델이 예측한 클래스(0 또는 1)을 저장
#
# # 🧪 10. 평가 지표 출력
# print(classification_report(y_test, y_pred))
# # 실제정답 y_test와 예측결과 y_pred를 비교해 분류 성능 지표들을 계산해 출력

In [3]:
# 📈 11. 교차검증 점수 (평가지표 5개)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# Straitified K-Fold 교차검증 설정
# 데이터를 5개의 Fold로 나누고, 각 Fold에서 클래스 비율(0 vs 1) 유지
# shuffle=True : 데이터를 섞어서 fold를 만들기 때문에 편향을 방지

# 평가지표 정의
scorers = {
    'f1': make_scorer(f1_score),
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    #'roc_auc': make_scorer(roc_auc_score, needs_proba=True)
    'roc_auc': make_scorer(roc_auc_score)
}


cv_results = {metric: cross_val_score(model, X_train, y_train, cv=cv, scoring=scorer).mean()
              for metric, scorer in scorers.items()}

# cross_val_score
# cv에서 정의한대로 훈련/검증을 5번 나눠 모델 학습 및 예측
# 각 fold마다 socrer 지표로 평가
# 평균값을 리턴 (.mean())


# 전체 메커니즘
# 1. StratifiedKFold : 훈련 데이터를 5개 폴드로 나눔 (클래스 비율 유지)
# 2. 각 폴드에 대해 모델 훈련 + 검증 반복
# 3. scorer : 평가 지표 정의 (f1, acc 등)
# 4. cross_val_score() : 해당 지표로 성능 측정
# 5. 각 지표의 평균값을 cv_results에 저장

# 🎯 테스트셋 F1 점수 추가
cv_results['f1(Test)'] = f1_score(y_test, y_pred)
# f1_socre() : 실제값(y_test)와 예측값(y_pred)를 비교해 F1-score를 계산


# 📊 결과 출력

print("📊 LR 교차검증 + 테스트셋 성능:\n")
for metric, score in cv_results.items():
    print(f"{metric:<10}: {score:.4f}")

from sklearn.model_selection import cross_val_predict

# # 확률 예측값을 얻기 위한 예측
# y_proba_cv = cross_val_predict(
#     model, X_train, y_train,
#     cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
#     method='predict_proba'
# )[:, 1]  # 양성 클래스 확률
#
# # AUC 수동 계산
# roc_auc_cv = roc_auc_score(y_train, y_proba_cv)
# print("Cross-validated ROC AUC:", roc_auc_cv)


📊 LR 교차검증 + 테스트셋 성능:

f1        : 0.7553
accuracy  : 0.7525
precision : 0.7474
recall    : 0.7637
roc_auc   : 0.7526
f1(Test)  : 0.7375


# 하이퍼 파라미터 튜닝 (precision, recall, f1)

In [112]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 🔧 LogisticRegression 하이퍼파라미터 후보 정의
param_grid = {
    'model__C': [0.01, 0.1, 1, 10],                 # 규제 강도 (C가 작을수록 규제 세짐)
    'model__penalty': ['l2'],                      # L2 정규화만 사용
    'model__solver': ['liblinear', 'lbfgs'],       # 최적화 알고리즘
    'model__multi_class': ['ovr']                  # 이진 분류에서 OVR 방식 고정
}

# ⚙️ 파이프라인: SMOTE → LogisticRegression
pipeline = Pipeline(steps=[
    ('smote', SMOTE(random_state=42)),                          # 클래스 불균형 처리
    ('model', LogisticRegression(max_iter=1000, random_state=42))  # 로지스틱 회귀 모델
])

# 📊 여러 평가지표 정의
scoring = {
    'precision': make_scorer(precision_score),                  # 정밀도
    'recall': make_scorer(recall_score),                        # 재현율
    'f1': make_scorer(f1_score),                                # F1 점수
    'accuracy': make_scorer(accuracy_score),                    # 정확도
    'roc_auc': make_scorer(roc_auc_score)                       # ROC AUC
}

# 📂 결과 저장용 딕셔너리
results = {}

# 🔁 Precision / Recall / F1 각각 기준으로 튜닝
for metric in ['precision', 'recall', 'f1']:
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring=scoring,             # 여러 지표 계산
        refit=metric,                # 최적 모델은 해당 metric 기준으로 선택
        cv=5,                        # 5-fold 교차검증
        verbose=0,
        n_jobs=-1,
        return_train_score=True
    )

    # 🏋️ 교차검증 수행
    grid.fit(X_train, y_train)

    # ✅ 최적 파라미터 정보
    best_index = grid.best_index_
    best_params = grid.best_params_

    # 📝 결과 저장
    results[metric] = {
        'best_params': best_params,
        'scores': {
            'Precision': grid.cv_results_['mean_test_precision'][best_index],
            'Recall': grid.cv_results_['mean_test_recall'][best_index],
            'F1 Score': grid.cv_results_['mean_test_f1'][best_index],
            'Accuracy': grid.cv_results_['mean_test_accuracy'][best_index],
            'ROC AUC': grid.cv_results_['mean_test_roc_auc'][best_index]
        }
    }

# 📢 결과 출력
for metric, content in results.items():
    print(f"\n🎯 [기준: {metric.upper()}]")
    print("Best Parameters:", content['best_params'])
    for score_name, val in content['scores'].items():
        print(f"{score_name:12}: {val:.4f}")



🎯 [기준: PRECISION]
Best Parameters: {'model__C': 0.01, 'model__multi_class': 'ovr', 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
Precision   : 0.7548
Recall      : 0.7748
F1 Score    : 0.7645
Accuracy    : 0.7612
ROC AUC     : 0.7612

🎯 [기준: RECALL]
Best Parameters: {'model__C': 0.01, 'model__multi_class': 'ovr', 'model__penalty': 'l2', 'model__solver': 'liblinear'}
Precision   : 0.7296
Recall      : 0.7992
F1 Score    : 0.7627
Accuracy    : 0.7515
ROC AUC     : 0.7515

🎯 [기준: F1]
Best Parameters: {'model__C': 0.01, 'model__multi_class': 'ovr', 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
Precision   : 0.7548
Recall      : 0.7748
F1 Score    : 0.7645
Accuracy    : 0.7612
ROC AUC     : 0.7612
